In [27]:
# ============================================
# LIBRERÍAS
# ============================================

import pandas as pd
import numpy as np

# Cargar el dataset limpio (proveniente del Notebook 3)
archivo = "data_clean.csv"
df = pd.read_csv(archivo)

print(f"Datos cargados correctamente desde '{archivo}'")
print(f"Total de registros: {len(df)}")
print(f"Columnas disponibles: {df.columns.tolist()}")

Datos cargados correctamente desde 'data_clean.csv'
Total de registros: 292
Columnas disponibles: ['id_paciente', 'nombre', 'rut', 'Edad', 'fecha_registro', 'id_medico', 'FamiliarPrimerGradoCC', 'FamiliarSegundoGradoCC', 'DiagnosticoPrevioCancer', 'Ejercicio', 'Alcohol', 'Mamografia', 'Menstruacion', 'PrimerHijo', 'riesgo', 'Edad_Original']


In [28]:
# Mostrar primeras filas para verificar estructura
print("\nPrimeras filas del dataset limpio:")
display(df.head())

# Mostrar tipos de datos
print("\nTipos de datos por columna:")
print(df.dtypes)


Primeras filas del dataset limpio:


,id_paciente,nombre,rut,Edad,fecha_registro,id_medico,FamiliarPrimerGradoCC,FamiliarSegundoGradoCC,DiagnosticoPrevioCancer,Ejercicio,Alcohol,Mamografia,Menstruacion,PrimerHijo,riesgo,Edad_Original
0,926,Bridie,53581753,3,09/10/2025,1,0.0,1,1,1,1.0,0,NaN,1.0,Alto,79
1,109,Bellina,88983746,0,09/10/2025,1,0.0,0,0,2,2.0,0,NaN,1.0,Alto,29
2,295,Starla,2366096,2,09/10/2025,1,1.0,0,0,2,0.0,0,NaN,NaN,Alto,54
3,976,Celine,90902307,3,09/10/2025,1,1.0,1,1,2,2.0,0,NaN,0.0,Moderado,79
4,735,Pietra,66433625,3,09/10/2025,1,1.0,1,1,1,2.0,1,NaN,1.0,Bajo,61



Tipos de datos por columna:
id_paciente                  int64
nombre                      object
rut                          int64
Edad                         int64
fecha_registro              object
id_medico                    int64
FamiliarPrimerGradoCC      float64
FamiliarSegundoGradoCC       int64
DiagnosticoPrevioCancer      int64
Ejercicio                    int64
Alcohol                    float64
Mamografia                   int64
Menstruacion               float64
PrimerHijo                 float64
riesgo                      object
Edad_Original                int64
dtype: object


In [29]:
# ============================================
# FUNCIÓN PARA DETECTAR OUTLIERS
# ============================================

def detectar_outliers(df, columnas):
    outliers = pd.DataFrame()
    for col in columnas:
        if col not in df.columns:
            print(f"Columna '{col}' no encontrada, se omitirá.")
            continue

        # Calcular cuartiles
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Definir límites
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR

        # Detectar valores fuera de rango
        mask = (df[col] < limite_inferior) | (df[col] > limite_superior)
        outliers[col] = mask
    return outliers

In [30]:
# ============================================
# APLICAR DETECCIÓN DE OUTLIERS
# ============================================

# Buscar columnas numéricas relevantes
columnas_numericas = [col for col in ['Edad', 'Puntaje'] if col in df.columns]

outliers = detectar_outliers(df, columnas_numericas)

# Mostrar resumen de outliers encontrados
print("\nCantidad de outliers detectados por columna:")
print(outliers.sum())

# Porcentaje sobre el total
porcentaje = (outliers.sum() / len(df)) * 100
print("\nPorcentaje de outliers por columna (%):")
print(porcentaje.round(2))


Cantidad de outliers detectados por columna:
Edad    0
dtype: int64

Porcentaje de outliers por columna (%):
Edad    0.0
dtype: float64


In [31]:
# ============================================
# MANEJO DE OUTLIERS
# ============================================

# Identificar filas con al menos un outlier
filas_con_outliers = outliers.any(axis=1)
cantidad_outliers = filas_con_outliers.sum()

print(f"\nTotal de registros con al menos un outlier: {cantidad_outliers}")

# Mostrar algunos ejemplos
print("\nEjemplos de registros con outliers:")
display(df[filas_con_outliers].head())

# Opción 1: Eliminar outliers (opcional)
# df_sin_outliers = df[~filas_con_outliers].copy()

# Opción 2: Reemplazar outliers por límites válidos
for col in columnas_numericas:
    if col not in df.columns: 
        continue
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], limite_inferior, limite_superior)

print("\nOutliers ajustados mediante recorte (clipping)")


Total de registros con al menos un outlier: 0

Ejemplos de registros con outliers:


,id_paciente,nombre,rut,Edad,fecha_registro,id_medico,FamiliarPrimerGradoCC,FamiliarSegundoGradoCC,DiagnosticoPrevioCancer,Ejercicio,Alcohol,Mamografia,Menstruacion,PrimerHijo,riesgo,Edad_Original



Outliers ajustados mediante recorte (clipping)


In [32]:
# ============================================
# GUARDAR VERSIÓN LIMPIA SIN OUTLIERS
# ============================================

df.to_csv("data_no_outliers.csv", index=False)
print("Archivo final guardado como 'data_no_outliers.csv'")

Archivo final guardado como 'data_no_outliers.csv'
